In [1]:
!pip install --upgrade tqdm

In [2]:
import numpy as np
import os
import random
import matplotlib.pyplot as plt
import seaborn as sns
import string
from tqdm import tqdm
import gc

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, precision_score, recall_score
from sklearn.pipeline import Pipeline

import glob
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.utils import load_img
from tensorflow.keras.applications import VGG16, VGG19
from tensorflow.keras.applications.vgg16 import preprocess_input as preprocess_input_vgg16
from tensorflow.keras.applications.vgg19 import preprocess_input as preprocess_input_vgg19
from tensorflow.keras.applications import ResNet50V2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Concatenate, Dense, Flatten, GlobalAveragePooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, optimizers, models, callbacks
from tensorflow.keras.utils import plot_model


2023-12-04 19:25:44.125156: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-04 19:25:44.125221: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-04 19:25:44.125247: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-04 19:25:44.133754: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


2023-12-04 19:25:51.533763: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


Evaluation functions for the models:

In [3]:
def plot_accuracy_loss(history):    
    acc = history.history['accuracy']
    val_acc = history.history['val_accuracy']

    loss = history.history['loss']
    val_loss = history.history['val_loss']

    epochs = range(1, len(acc) + 1)

    plt.figure(figsize = (10, 7))

    plt.subplot(1,2,1)
    plt.plot(epochs, acc, c = 'b', label = 'Training Accuracy')
    plt.plot(epochs, val_acc, c = 'g', label = 'Validation Accuracy')
    plt.title('Training vs. Validation Accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend()

    plt.subplot(1,2,2)
    plt.plot(epochs, loss, c = 'b', label = 'Training Loss')
    plt.plot(epochs, val_loss, c = 'g', label = 'Validation Loss')
    plt.title('Training vs. Validation Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()

    plt.show()

In [4]:
def show_accuracy_loss(model, test_generator):
    print('---------------Evaluation Against Test Data---------------------')
    eval_loss, eval_acc = model.evaluate(test_generator)
    print('Evaluation Loss: {:.4f}, Evaluation Accuracy: {:.2f}'.format(eval_loss, eval_acc * 100))
    print('----------------------------------------------------------------')

In [5]:
def show_predictions(model, color_data, depth_data, labels, class_names, model_name):
    if model_name == 'vgg16':
        color_data = preprocess_input_vgg16(color_data)
    elif model_name == 'vgg19':
        color_data = preprocess_input_vgg19(color_data)
    depth_data = depth_data / 255.0  # Assuming depth values are in the range [0, 255]

    # Make predictions
    predictions = model.predict([color_data, depth_data])
    predicted_labels = np.argmax(predictions, axis=1)

    # Calculate accuracy
    accuracy = accuracy_score(labels, predicted_labels)
    print(f"Accuracy: {accuracy:.2%}")

    # Generate a classification report
    report = classification_report(labels, predicted_labels, target_names=class_names)
    print(report)

    # Generate a confusion matrix
    cm = confusion_matrix(labels, predicted_labels)

    # Plot confusion matrix
    plt.figure(figsize=(8, 6))
    plt.imshow(cm, interpolation="nearest", cmap=plt.cm.Blues)
    plt.title("Confusion Matrix")
    plt.colorbar()
    tick_marks = np.arange(len(class_names))
    plt.xticks(tick_marks, class_names, rotation=45)
    plt.yticks(tick_marks, class_names)
    plt.xlabel("Predicted")
    plt.ylabel("True")
    plt.show()

# Define class names based on your label mapping
class_names = ["a", "b", "c", "d", "e", "f", "g", "h", "i", "k","l", "m", "n", "o", "p", "q", "r", "s", "t", "u","v", "w", "x", "y"], 

# Constants:

In [6]:
# Constants
IMAGE_SIZE = (224, 224)
DATA_PATH = "./augmented_images"
BATCH_SIZE = 64
LABEL_CLASS = 24
LEARNINGRATE = 1e-5
WEIGHTS = 'imagenet'
SEED = 99
EPOCHS = 15

In [7]:
len(os.listdir(os.path.join(DATA_PATH, "A")))

25

In [8]:
os.listdir(os.path.join(DATA_PATH, "A"))

['v',
 'p',
 'd',
 't',
 'l',
 'e',
 'q',
 'u',
 'm',
 'i',
 'h',
 'g',
 'c',
 'b',
 'o',
 'f',
 'n',
 'w',
 'x',
 '.ipynb_checkpoints',
 's',
 'a',
 'r',
 'y',
 'k']

In [9]:
def load_data_from_innermost_folder(folder_path, target_size=IMAGE_SIZE):
    color_images = []
    depth_maps = []
    labels = []
    for sub_folder in sorted(os.listdir(folder_path)):
        if sub_folder == '.ipynb_checkpoints':
            continue
        print("sub_folder :", sub_folder)
        sub_folder_path = os.path.join(folder_path, sub_folder)
        
        for file_name in os.listdir(sub_folder_path):
            if "color" in file_name:
                # print("Beginning pairing for -> ", file_name)
                color_img_name = os.path.join(sub_folder_path, file_name)
                depth_img_name = os.path.join(sub_folder_path, file_name.replace("color", "depth"))

                color_img = load_img(color_img_name, target_size=target_size)
                depth_img = load_img(depth_img_name, target_size=target_size, color_mode='grayscale')
                
                color_images.append(img_to_array(color_img))
                depth_maps.append(img_to_array(depth_img))
                labels.append(ord(sub_folder) - ord('a'))
                
    print("\n\n#####################\nPAIRING ENDED\n#####################")
    
    return np.array(color_images), np.array(depth_maps), np.array(labels)

# def load_data_from_innermost_folder(folder_path, target_size=IMAGE_SIZE):
#     color_images = []
#     depth_maps = []
#     labels = []

#     # Create a dictionary to map class names to labels
#     class_names = ["a", "b", "c", "d", "e", "f", "g", "h", "i", "k","l", "m", "n", "o", "p", "q", "r", "s", "t", "u","v", "w", "x", "y"]
#     class_to_label = {class_name: label for label, class_name in enumerate(class_names)}

#     for sub_folder in sorted(os.listdir(folder_path)):
#         sub_folder_path = os.path.join(folder_path, sub_folder)
        
#         for file_name in os.listdir(sub_folder_path):
#             if "color" in file_name:
#                 print("pairing for --> ", file_name)
#                 color_img_name = os.path.join(sub_folder_path, file_name)
#                 depth_img_name = os.path.join(sub_folder_path, file_name.replace("color", "depth"))
                
#                 color_img = load_img(color_img_name, target_size=target_size)
#                 depth_img = load_img(depth_img_name, target_size=target_size, color_mode='grayscale')
                
#                 color_images.append(img_to_array(color_img))
#                 depth_maps.append(img_to_array(depth_img))
#                 labels.append(class_to_label[sub_folder])  # Map class name to label using the dictionary

#     return np.array(color_images), np.array(depth_maps), np.array(labels)


def load_paired_data(base_path, target_size=IMAGE_SIZE):
    color_images = []
    depth_maps = []
    labels = []

    for folder in sorted(os.listdir(base_path)):
        if folder != ".ipynb_checkpoints":
            folder_path = os.path.join(base_path, folder)
            
            print("Beginning data pairing...")
            
            color_folder_data, depth_folder_data, folder_labels = load_data_from_innermost_folder(folder_path, target_size)
    
            print("Data pairing completed.")
            return color_folder_data, depth_folder_data, folder_labels
        
    #         color_images = np.concatenate((color_images, color_folder_data))
    #         depth_maps = np.concatenate((depth_maps, depth_folder_data))
    #         labels = np.concatenate((labels, folder_labels))
    #         # color_images.extend(color_folder_data)
    #         # depth_maps.extend(depth_folder_data)
    #         # labels.extend(folder_labels)
    # # return np.array(color_images), np.array(depth_maps), np.array(labels)
    # return color_images, depth_maps, labels


In [10]:
sorted(os.listdir(DATA_PATH))

['.ipynb_checkpoints', 'A', 'B', 'C', 'D', 'E']

In [11]:
# color_data, depth_data, labels = load_paired_data(DATA_PATH)
color_folder_data, depth_folder_data, folder_labels = load_paired_data(DATA_PATH)

Beginning data pairing...
sub_folder : a


sub_folder : b


sub_folder : c


sub_folder : d


sub_folder : e


...

sub_folder : w


sub_folder : x


sub_folder : y




#####################
PAIRING ENDED
#####################


Data pairing completed.


In [12]:
# color_folder_data.shape
folder_labels.shape

(14400,)

In [13]:
folder_labels[0]

0

In [14]:
np.unique(folder_labels).shape

(24,)

In [15]:
print("creating numpy array...")
color_images = np.empty((0,) + IMAGE_SIZE + (3,), dtype=np.float32)
print("color numpy array created")

depth_maps = np.empty((0,) + IMAGE_SIZE + (1,), dtype=np.float32)
print("depth numpy array created")

labels = np.empty((0,), dtype=np.int32)
print("label numpy array created")


creating numpy array...
color numpy array created
depth numpy array created
label numpy array created


In [16]:
folder_labels.shape

(14400,)

In [17]:
print("concat numpy array...")

color_data = np.concatenate((color_images, color_folder_data))
print("color data done")

depth_data = np.concatenate((depth_maps, depth_folder_data))
print("depth data done")

labels = np.concatenate((labels, folder_labels))
print("labels data done")


concat numpy array...


color data done


depth data done
labels data done


In [18]:
color_data.shape, depth_data.shape, labels.shape

((14400, 224, 224, 3), (14400, 224, 224, 1), (14400,))

In [19]:
np.unique(labels)

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24])

In [20]:
labels[labels==24] = 9

In [21]:
labels = tf.keras.utils.to_categorical(labels, 24)

In [22]:
labels.shape

(14400, 24)

## First train test split

In [23]:
print("creating x indexing...")

X_index = np.arange(0, 14400)
X_train, X_test, labels_train, labels_test = train_test_split(X_index, labels, test_size=0.3, random_state=42)

print("indexing completed!")

creating x indexing...
indexing completed!


In [24]:
labels_train.shape

(10080, 24)

In [25]:
print("Beginning : color_train")

color_train = color_data[X_train]
print("Ended : color_train")

Beginning : color_train


Ended : color_train


In [26]:
# color_temp = color_data[X_test]

print("Beginning : color_test")
color_test = color_data[X_test]
del color_data
print("Ended : color_test, color_data deleted")

Beginning : color_test


Ended : color_test, color_data deleted


In [27]:
print("Beginning : depth_train")
depth_train = depth_data[X_train]
print("Ended : depth_train")

Beginning : depth_train


Ended : depth_train


In [28]:
# depth_temp = depth_data[X_test]
print("Beginning : depth_test")

depth_test = depth_data[X_test]
del depth_data
print("Ended : depth_test, depth_data deleted")

Beginning : depth_test
Ended : depth_test, depth_data deleted


In [29]:
gc.collect()

0

## Second train test split

In [30]:
# X_index2 = np.arange(0, )
# X_train2, X_test2, y_train2, y_test2 = train_test_split(X_index2, labels, test_size=0.3, random_state=42)

In [31]:
# color_val = color_temp[X_train2]
# color_test = color_temp[X_test2]
# depth_val = depth_temp[X_train2]
# depth_test = depth_temp[X_test2]
# labels_val = labels_temp[y_train2]
# labels_test = labels_temp[y_test2]

In [32]:
# # Split data with tqdm progress bar
# print("Beginning splitting into train test...")
# with tqdm(total=len(color_data), desc="Splitting Data") as pbar:
#     pbar.update(0)
#     color_train, color_temp, depth_train, depth_temp, labels_train, labels_temp = train_test_split(
#         color_data, depth_data, labels, test_size=0.3, random_state=42
#     )
#     pbar.update(len(color_train))

In [33]:
# # Split data with tqdm progress bar
# print("Beginning splitting into train test...")
# with tqdm(total=len(color_temp), desc="Splitting Data") as pbar:    
#     print("Beginning second split...")
#     color_val, color_test, depth_val, depth_test, labels_val, labels_test = train_test_split(
#         color_temp, depth_temp, labels_temp, test_size=0.5, random_state=42
#     )
#     pbar.update(len(color_val) + len(color_test))

# VGG 16 Model

### Preloaded model

In [34]:
# vgg16_preload_model = keras.models.load_model('vgg_16_model.keras')

In [35]:
# predicted_labels = vgg16_preload_model.predict([color_test, depth_test])
# predicted_classes = np.argmax(predicted_labels, axis=1)
# true_classes = np.argmax(labels_test, axis=1)
# try:
#     accuracy = accuracy_score(true_classes, predicted_classes)
#     print("Accuracy:", accuracy)
# except:
#     print("accuracy fail")

# try:
#     precision = precision_score(true_classes, predicted_classes, average='macro')
#     print("Precision:", precision)
# except:
#     print("precision fail")

# try:
#     recall = recall_score(true_classes, predicted_classes, average='macro')
#     print("Recall:", recall)
# except:
#     print("recall fail")

# try:
#     f1 = f1_score(true_classes, predicted_classes, average='macro')
#     print("F1 Score:", f1)
# except:
#     print("f1 fail")

# try:
#     class_report = classification_report(true_classes, predicted_classes)
#     print("Classification Report:")
#     print(class_report)
# except:
#     print("class report fail")

### Model Constructor

In [36]:
def build_vgg_16_model(
    image_size,
    label_class,
    weights,
):
    # Loading the VGG models with unique names
    vgg_color = VGG16(include_top=False, weights=weights, input_shape=(image_size[0], image_size[1], 3))
    for layer in vgg_color.layers:
        layer._name = layer.name + "_color"   # rename the layers by appending "_color"

    vgg_depth = VGG16(include_top=False, weights=None, input_shape=(image_size[0], image_size[1], 1))
    for layer in vgg_depth.layers:
        layer._name = layer.name + "_depth"   # rename the layers by appending "_depth"

    # Extract features from both models
    color_features = vgg_color.output
    depth_features = vgg_depth.output

    # Concatenate the extracted features
    concatenated = Concatenate()([color_features, depth_features])

    # Flatten and add a dense layer (assuming a simple classification task)
    flattened = Flatten()(concatenated)

        # Additional dense layers and output layer
    X = layers.Dense(512, activation='relu')(flattened)
    X = layers.Dropout(0.4)(X)
    X = layers.BatchNormalization()(X)
    X = layers.Dense(512, activation='relu')(X)
    X = layers.Dropout(0.3)(X)
    X = layers.BatchNormalization()(X)

    outputs = Dense(label_class, activation='softmax')(X)
    model = Model(inputs=[vgg_color.input, vgg_depth.input], outputs=outputs)
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])
    return model

In [37]:
# def build_vgg_16_model(
#     image_size,
#     label_class,
#     weights,
# ):
#     # Loading the VGG models with unique names
#     vgg_color = VGG16(include_top=False, weights=weights, input_shape=(image_size[0], image_size[1], 3))
#     for layer in vgg_color.layers:
#         layer._name = layer.name + "_color"   # rename the layers by appending "_color"

#     vgg_depth = VGG16(include_top=False, weights=None, input_shape=(image_size[0], image_size[1], 1))
#     for layer in vgg_depth.layers:
#         layer._name = layer.name + "_depth"   # rename the layers by appending "_depth"

#     # Extract features from both models
#     color_features = vgg_color.output
#     depth_features = vgg_depth.output

#     # Concatenate the extracted features
#     concatenated = Concatenate()([color_features, depth_features])

#     # Flatten and add a dense layer (assuming a simple classification task)
#     flattened = Flatten()(concatenated)

#     #     # Additional dense layers and output layer
#     # X = layers.Dense(512, activation='relu')(flattened)
#     # X = layers.Dropout(0.4)(X)
#     # X = layers.BatchNormalization()(X)
#     # X = layers.Dense(512, activation='relu')(X)
#     # X = layers.Dropout(0.3)(X)
#     # X = layers.BatchNormalization()(X)

#     outputs = Dense(label_class, activation='softmax')(flattened)
#     model = Model(inputs=[vgg_color.input, vgg_depth.input], outputs=outputs)
#     model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])
#     return model

### DONE- VGG16 (batch size = 64; epoch = 15) => DEFAULT

In [38]:
print("Beginning VGG16 training..")

vgg16 = build_vgg_16_model(
    image_size=IMAGE_SIZE,
    label_class=LABEL_CLASS,
    weights=WEIGHTS,
)

vgg16.fit(
    [color_train, depth_train],
    labels_train,
    # validation_data=([color_val, depth_val], labels_val),
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    verbose=1,
    callbacks=[
        callbacks.EarlyStopping(patience=3),
        callbacks.ModelCheckpoint("vgg_16_model_best.h5", save_best_only=True),  # Save the entire model
        callbacks.ReduceLROnPlateau(monitor='val_loss', patience= 2, verbose= 1),      
    ],
)

print("Ended VGG16 training")

# Save the entire model
# vgg16.save("vgg_16_model.keras")
# print("VGG16 model saved!")

predicted_labels = vgg16.predict([color_test, depth_test])
predicted_classes = np.argmax(predicted_labels, axis=1)
true_classes = np.argmax(labels_test, axis=1)
try:
    accuracy = accuracy_score(true_classes, predicted_classes)
    print("Accuracy:", accuracy)
except:
    print("accuracy fail")

try:
    precision = precision_score(true_classes, predicted_classes, average='macro')
    print("Precision:", precision)
except:
    print("precision fail")

try:
    recall = recall_score(true_classes, predicted_classes, average='macro')
    print("Recall:", recall)
except Exception as e:
    print("recall fail")
    print(e)

try:
    f1 = f1_score(true_classes, predicted_classes, average='macro')
    print("F1 Score:", f1)
except Exception as e:
    print("f1 fail")
    print(e)

try:
    class_report = classification_report(true_classes, predicted_classes)
    print("Classification Report:")
    print(class_report)
except:
    print("class report fail")


Beginning VGG16 training..


2023-12-04 19:26:21.538068: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1886] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 22460 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3090, pci bus id: 0000:41:00.0, compute capability: 8.6


Epoch 1/15


2023-12-04 19:26:30.513314: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:442] Loaded cuDNN version 8600


2023-12-04 19:26:33.454046: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7fd0f2ca8340 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-12-04 19:26:33.454086: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 3090, Compute Capability 8.6
2023-12-04 19:26:33.459431: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-12-04 19:26:33.569419: I ./tensorflow/compiler/jit/device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


...

133/135 [============================>.] - ETA: 0s

134/135 [============================>.] - ETA: 0s

135/135 [==============================] - ETA: 0s

135/135 [==============================] - 9s 67ms/step


Accuracy: 0.9592592592592593
Precision: 0.9601441725936689
Recall: 0.959372275468211
f1 fail
name 'f1_score' is not defined
Classification Report:
              precision    recall  f1-score   support

           0       0.95      0.97      0.96       189
           1       0.97      0.99      0.98       175
           2       0.97      1.00      0.99       187
           3       0.95      0.98      0.97       172
           4       0.97      0.96      0.96       182
           5       0.99      0.97      0.98       186
           6       0.88      0.98      0.93       196
           7       0.98      0.85      0.91       184
           8       0.95      0.99      0.97       166
           9       0.99      0.98      0.98       187
          10       0.98      0.96      0.97       167
          11       0.99      0.98      0.99       177
          12       0.95      0.97      0.96       157
          13       0.94      0.91      0.92       181
          14       0.97      0.96      0.9

### DONE- VGG16 (batch size = 32; epoch = 15)

In [39]:
# print("Beginning VGG16 training..")

# vgg16 = build_vgg_16_model(
#     image_size=IMAGE_SIZE,
#     label_class=LABEL_CLASS,
#     weights=WEIGHTS,
# )

# vgg16.fit(
#     [color_train, depth_train],
#     labels_train,
#     # validation_data=([color_val, depth_val], labels_val),
#     epochs=15,
#     batch_size=32,
#     verbose=1,
#     callbacks=[
#         callbacks.EarlyStopping(patience=3),
#         callbacks.ModelCheckpoint("vgg_16_model_best.h5", save_best_only=True),  # Save the entire model
#         callbacks.ReduceLROnPlateau(monitor='val_loss', patience= 2, verbose= 1),      
#     ],
# )

# print("Ended VGG16 training")

# # Save the entire model
# # vgg16.save("vgg_16_model.keras")
# # print("VGG16 model saved!")

# predicted_labels = vgg16.predict([color_test, depth_test])
# predicted_classes = np.argmax(predicted_labels, axis=1)
# true_classes = np.argmax(labels_test, axis=1)
# try:
#     accuracy = accuracy_score(true_classes, predicted_classes)
#     print("Accuracy:", accuracy)
# except:
#     print("accuracy fail")

# try:
#     precision = precision_score(true_classes, predicted_classes, average='macro')
#     print("Precision:", precision)
# except:
#     print("precision fail")

# try:
#     recall = recall_score(true_classes, predicted_classes, average='macro')
#     print("Recall:", recall)
# except Exception as e:
#     print("recall fail")
#     print(e)

# try:
#     f1 = f1_score(true_classes, predicted_classes, average='macro')
#     print("F1 Score:", f1)
# except Exception as e:
#     print("f1 fail")
#     print(e)

# try:
#     class_report = classification_report(true_classes, predicted_classes)
#     print("Classification Report:")
#     print(class_report)
# except:
#     print("class report fail")


### DONE- VGG16 (batch size = 64; epoch = 30)

In [40]:
# print("Beginning VGG16 training..")

# vgg16 = build_vgg_16_model(
#     image_size=IMAGE_SIZE,
#     label_class=LABEL_CLASS,
#     weights=WEIGHTS,
# )

# vgg16.fit(
#     [color_train, depth_train],
#     labels_train,
#     # validation_data=([color_val, depth_val], labels_val),
#     epochs=30,
#     batch_size=64,
#     verbose=1,
#     callbacks=[
#         callbacks.EarlyStopping(patience=3),
#         callbacks.ModelCheckpoint("vgg_16_model_best.h5", save_best_only=True),  # Save the entire model
#         callbacks.ReduceLROnPlateau(monitor='val_loss', patience= 2, verbose= 1),      
#     ],
# )

# print("Ended VGG16 training")

# # Save the entire model
# # vgg16.save("vgg_16_model.keras")
# # print("VGG16 model saved!")

# predicted_labels = vgg16.predict([color_test, depth_test])
# predicted_classes = np.argmax(predicted_labels, axis=1)
# true_classes = np.argmax(labels_test, axis=1)
# try:
#     accuracy = accuracy_score(true_classes, predicted_classes)
#     print("Accuracy:", accuracy)
# except:
#     print("accuracy fail")

# try:
#     precision = precision_score(true_classes, predicted_classes, average='macro')
#     print("Precision:", precision)
# except:
#     print("precision fail")

# try:
#     recall = recall_score(true_classes, predicted_classes, average='macro')
#     print("Recall:", recall)
# except Exception as e:
#     print("recall fail")
#     print(e)

# try:
#     f1 = f1_score(true_classes, predicted_classes, average='macro')
#     print("F1 Score:", f1)
# except Exception as e:
#     print("f1 fail")
#     print(e)

# try:
#     class_report = classification_report(true_classes, predicted_classes)
#     print("Classification Report:")
#     print(class_report)
# except:
#     print("class report fail")


### DONE- VGG16 (batch size = 64; epoch = 45)

In [41]:
# print("Beginning VGG16 training..")

# vgg16 = build_vgg_16_model(
#     image_size=IMAGE_SIZE,
#     label_class=LABEL_CLASS,
#     weights=WEIGHTS,
# )

# vgg16.fit(
#     [color_train, depth_train],
#     labels_train,
#     # validation_data=([color_val, depth_val], labels_val),
#     epochs=45,
#     batch_size=64,
#     verbose=1,
#     callbacks=[
#         callbacks.EarlyStopping(patience=3),
#         callbacks.ModelCheckpoint("vgg_16_model_best.h5", save_best_only=True),  # Save the entire model
#         callbacks.ReduceLROnPlateau(monitor='val_loss', patience= 2, verbose= 1),      
#     ],
# )

# print("Ended VGG16 training")

# # Save the entire model
# # vgg16.save("vgg_16_model.keras")
# # print("VGG16 model saved!")

# predicted_labels = vgg16.predict([color_test, depth_test])
# predicted_classes = np.argmax(predicted_labels, axis=1)
# true_classes = np.argmax(labels_test, axis=1)
# try:
#     accuracy = accuracy_score(true_classes, predicted_classes)
#     print("Accuracy:", accuracy)
# except:
#     print("accuracy fail")

# try:
#     precision = precision_score(true_classes, predicted_classes, average='macro')
#     print("Precision:", precision)
# except:
#     print("precision fail")

# try:
#     recall = recall_score(true_classes, predicted_classes, average='macro')
#     print("Recall:", recall)
# except Exception as e:
#     print("recall fail")
#     print(e)

# try:
#     # f1 = f1_score(true_classes, predicted_classes, average='macro')
#     f1 = 2 * ((precision * recall) / (precision + recall))
#     print("F1 Score:", f1)
# except Exception as e:
#     print("f1 fail")
#     print(e)

# try:
#     class_report = classification_report(true_classes, predicted_classes)
#     print("Classification Report:")
#     print(class_report)
# except:
#     print("class report fail")


In [42]:
# def build_vgg_16_model_nadam(
#     image_size,
#     label_class,
#     weights,
# ):
#     # Loading the VGG models with unique names
#     vgg_color = VGG16(include_top=False, weights=weights, input_shape=(image_size[0], image_size[1], 3))
#     for layer in vgg_color.layers:
#         layer._name = layer.name + "_color"   # rename the layers by appending "_color"

#     vgg_depth = VGG16(include_top=False, weights=None, input_shape=(image_size[0], image_size[1], 1))
#     for layer in vgg_depth.layers:
#         layer._name = layer.name + "_depth"   # rename the layers by appending "_depth"

#     # Extract features from both models
#     color_features = vgg_color.output
#     depth_features = vgg_depth.output

#     # Concatenate the extracted features
#     concatenated = Concatenate()([color_features, depth_features])

#     # Flatten and add a dense layer (assuming a simple classification task)
#     flattened = Flatten()(concatenated)

#         # Additional dense layers and output layer
#     X = layers.Dense(512, activation='relu')(flattened)
#     X = layers.Dropout(0.4)(X)
#     X = layers.BatchNormalization()(X)
#     X = layers.Dense(512, activation='relu')(X)
#     X = layers.Dropout(0.3)(X)
#     X = layers.BatchNormalization()(X)

#     outputs = Dense(label_class, activation='softmax')(X)
#     model = Model(inputs=[vgg_color.input, vgg_depth.input], outputs=outputs)
#     model.compile(optimizer='nadam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])
#     return model

## DONE - VGG16 VGG16 (batch size = 64; epoch = 45, Optimizer = nadam)

In [43]:
# def build_vgg_16_model_nadam(
#     image_size,
#     label_class,
#     weights,
# ):
#     # Loading the VGG models with unique names
#     vgg_color = VGG16(include_top=False, weights=weights, input_shape=(image_size[0], image_size[1], 3))
#     for layer in vgg_color.layers:
#         layer._name = layer.name + "_color"   # rename the layers by appending "_color"

#     vgg_depth = VGG16(include_top=False, weights=None, input_shape=(image_size[0], image_size[1], 1))
#     for layer in vgg_depth.layers:
#         layer._name = layer.name + "_depth"   # rename the layers by appending "_depth"

#     # Extract features from both models
#     color_features = vgg_color.output
#     depth_features = vgg_depth.output

#     # Concatenate the extracted features
#     concatenated = Concatenate()([color_features, depth_features])

#     # Flatten and add a dense layer (assuming a simple classification task)
#     flattened = Flatten()(concatenated)

#         # Additional dense layers and output layer
#     X = layers.Dense(512, activation='relu')(flattened)
#     X = layers.Dropout(0.4)(X)
#     X = layers.BatchNormalization()(X)
#     X = layers.Dense(512, activation='relu')(X)
#     X = layers.Dropout(0.3)(X)
#     X = layers.BatchNormalization()(X)

#     outputs = Dense(label_class, activation='softmax')(X)
#     model = Model(inputs=[vgg_color.input, vgg_depth.input], outputs=outputs)
#     model.compile(optimizer='nadam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])
#     return model

In [44]:
# print("Beginning VGG16 training..")

# vgg16 = build_vgg_16_model_nadam(
#     image_size=IMAGE_SIZE,
#     label_class=LABEL_CLASS,
#     weights=WEIGHTS,
# )

# vgg16.fit(
#     [color_train, depth_train],
#     labels_train,
#     # validation_data=([color_val, depth_val], labels_val),
#     epochs=45,
#     batch_size=64,
#     verbose=0,
#     callbacks=[
#         callbacks.EarlyStopping(patience=3),
#         callbacks.ModelCheckpoint("vgg_16_model_best.h5", save_best_only=True),  # Save the entire model
#         callbacks.ReduceLROnPlateau(monitor='val_loss', patience= 2, verbose= 1),      
#     ],
# )

# print("Ended VGG16 training")

# # Save the entire model
# # vgg16.save("vgg_16_model.keras")
# # print("VGG16 model saved!")

# predicted_labels = vgg16.predict([color_test, depth_test])
# predicted_classes = np.argmax(predicted_labels, axis=1)
# true_classes = np.argmax(labels_test, axis=1)
# try:
#     accuracy = accuracy_score(true_classes, predicted_classes)
#     print("Accuracy:", accuracy)
# except:
#     print("accuracy fail")

# try:
#     precision = precision_score(true_classes, predicted_classes, average='macro')
#     print("Precision:", precision)
# except:
#     print("precision fail")

# try:
#     recall = recall_score(true_classes, predicted_classes, average='macro')
#     print("Recall:", recall)
# except Exception as e:
#     print("recall fail")
#     print(e)

# try:
#     # f1 = f1_score(true_classes, predicted_classes, average='macro')
#     f1 = 2 * ((precision * recall) / (precision + recall))
#     print("F1 Score:", f1)
# except Exception as e:
#     print("f1 fail")
#     print(e)

# try:
#     class_report = classification_report(true_classes, predicted_classes)
#     print("Classification Report:")
#     print(class_report)
# except:
#     print("class report fail")


## DONE - VGG16 VGG16 (batch size = 64; epoch = 45, Optimizer = SGD)

In [45]:
# def build_vgg_16_model_sgd(
#     image_size,
#     label_class,
#     weights,
# ):
#     # Loading the VGG models with unique names
#     vgg_color = VGG16(include_top=False, weights=weights, input_shape=(image_size[0], image_size[1], 3))
#     for layer in vgg_color.layers:
#         layer._name = layer.name + "_color"   # rename the layers by appending "_color"

#     vgg_depth = VGG16(include_top=False, weights=None, input_shape=(image_size[0], image_size[1], 1))
#     for layer in vgg_depth.layers:
#         layer._name = layer.name + "_depth"   # rename the layers by appending "_depth"

#     # Extract features from both models
#     color_features = vgg_color.output
#     depth_features = vgg_depth.output

#     # Concatenate the extracted features
#     concatenated = Concatenate()([color_features, depth_features])

#     # Flatten and add a dense layer (assuming a simple classification task)
#     flattened = Flatten()(concatenated)

#         # Additional dense layers and output layer
#     X = layers.Dense(512, activation='relu')(flattened)
#     X = layers.Dropout(0.4)(X)
#     X = layers.BatchNormalization()(X)
#     X = layers.Dense(512, activation='relu')(X)
#     X = layers.Dropout(0.3)(X)
#     X = layers.BatchNormalization()(X)

#     outputs = Dense(label_class, activation='softmax')(X)
#     model = Model(inputs=[vgg_color.input, vgg_depth.input], outputs=outputs)
#     model.compile(optimizer='SGD', loss='categorical_crossentropy', metrics=['categorical_accuracy'])
#     return model

In [46]:
# print("Beginning VGG16 training..")

# vgg16 = build_vgg_16_model_sgd(
#     image_size=IMAGE_SIZE,
#     label_class=LABEL_CLASS,
#     weights=WEIGHTS,
# )

# vgg16.fit(
#     [color_train, depth_train],
#     labels_train,
#     # validation_data=([color_val, depth_val], labels_val),
#     epochs=45,
#     batch_size=64,
#     verbose=0,
#     callbacks=[
#         callbacks.EarlyStopping(patience=3),
#         callbacks.ModelCheckpoint("vgg_16_model_best.h5", save_best_only=True),  # Save the entire model
#         callbacks.ReduceLROnPlateau(monitor='val_loss', patience= 2, verbose= 1),      
#     ],
# )

# print("Ended VGG16 training")

# # Save the entire model
# # vgg16.save("vgg_16_model.keras")
# # print("VGG16 model saved!")

# predicted_labels = vgg16.predict([color_test, depth_test])
# predicted_classes = np.argmax(predicted_labels, axis=1)
# true_classes = np.argmax(labels_test, axis=1)
# try:
#     accuracy = accuracy_score(true_classes, predicted_classes)
#     print("Accuracy:", accuracy)
# except:
#     print("accuracy fail")

# try:
#     precision = precision_score(true_classes, predicted_classes, average='macro')
#     print("Precision:", precision)
# except:
#     print("precision fail")

# try:
#     recall = recall_score(true_classes, predicted_classes, average='macro')
#     print("Recall:", recall)
# except Exception as e:
#     print("recall fail")
#     print(e)

# try:
#     # f1 = f1_score(true_classes, predicted_classes, average='macro')
#     f1 = 2 * ((precision * recall) / (precision + recall))
#     print("F1 Score:", f1)
# except Exception as e:
#     print("f1 fail")
#     print(e)

# try:
#     class_report = classification_report(true_classes, predicted_classes)
#     print("Classification Report:")
#     print(class_report)
# except:
#     print("class report fail")


## DONE - VGG16 VGG16 (batch size = 64; epoch = 15, Optimizer = RMSprop)

In [47]:
# def build_vgg_16_model_rmsprop(
#     image_size,
#     label_class,
#     weights,
# ):
#     # Loading the VGG models with unique names
#     vgg_color = VGG16(include_top=False, weights=weights, input_shape=(image_size[0], image_size[1], 3))
#     for layer in vgg_color.layers:
#         layer._name = layer.name + "_color"   # rename the layers by appending "_color"

#     vgg_depth = VGG16(include_top=False, weights=None, input_shape=(image_size[0], image_size[1], 1))
#     for layer in vgg_depth.layers:
#         layer._name = layer.name + "_depth"   # rename the layers by appending "_depth"

#     # Extract features from both models
#     color_features = vgg_color.output
#     depth_features = vgg_depth.output

#     # Concatenate the extracted features
#     concatenated = Concatenate()([color_features, depth_features])

#     # Flatten and add a dense layer (assuming a simple classification task)
#     flattened = Flatten()(concatenated)

#         # Additional dense layers and output layer
#     X = layers.Dense(512, activation='relu')(flattened)
#     X = layers.Dropout(0.4)(X)
#     X = layers.BatchNormalization()(X)
#     X = layers.Dense(512, activation='relu')(X)
#     X = layers.Dropout(0.3)(X)
#     X = layers.BatchNormalization()(X)

#     outputs = Dense(label_class, activation='softmax')(X)
#     model = Model(inputs=[vgg_color.input, vgg_depth.input], outputs=outputs)
#     model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['categorical_accuracy'])
#     return model

In [48]:
# print("Beginning VGG16 training..")

# vgg16 = build_vgg_16_model_rmsprop(
#     image_size=IMAGE_SIZE,
#     label_class=LABEL_CLASS,
#     weights=WEIGHTS,
# )

# vgg16.fit(
#     [color_train, depth_train],
#     labels_train,
#     # validation_data=([color_val, depth_val], labels_val),
#     epochs=15,
#     batch_size=64,
#     verbose=0,
#     callbacks=[
#         callbacks.EarlyStopping(patience=3),
#         callbacks.ModelCheckpoint("vgg_16_model_best.h5", save_best_only=True),  # Save the entire model
#         callbacks.ReduceLROnPlateau(monitor='val_loss', patience= 2, verbose= 1),      
#     ],
# )

# print("Ended VGG16 training")

# # Save the entire model
# # vgg16.save("vgg_16_model.keras")
# # print("VGG16 model saved!")

# predicted_labels = vgg16.predict([color_test, depth_test])
# predicted_classes = np.argmax(predicted_labels, axis=1)
# true_classes = np.argmax(labels_test, axis=1)
# try:
#     accuracy = accuracy_score(true_classes, predicted_classes)
#     print("Accuracy:", accuracy)
# except:
#     print("accuracy fail")

# try:
#     precision = precision_score(true_classes, predicted_classes, average='macro')
#     print("Precision:", precision)
# except:
#     print("precision fail")

# try:
#     recall = recall_score(true_classes, predicted_classes, average='macro')
#     print("Recall:", recall)
# except Exception as e:
#     print("recall fail")
#     print(e)

# try:
#     # f1 = f1_score(true_classes, predicted_classes, average='macro')
#     f1 = 2 * ((precision * recall) / (precision + recall))
#     print("F1 Score:", f1)
# except Exception as e:
#     print("f1 fail")
#     print(e)

# try:
#     class_report = classification_report(true_classes, predicted_classes)
#     print("Classification Report:")
#     print(class_report)
# except:
#     print("class report fail")


# VGG 19

### Preloaded Model

In [49]:
# vgg19_preload_model = keras.models.load_model('vgg_19_model.keras')

In [50]:
# predicted_labels = vgg19_preload_model.predict([color_test, depth_test])
# predicted_classes = np.argmax(predicted_labels, axis=1)
# true_classes = np.argmax(labels_test, axis=1)
# try:
#     accuracy = accuracy_score(true_classes, predicted_classes)
#     print("Accuracy:", accuracy)
# except:
#     print("accuracy fail")

# try:
#     precision = precision_score(true_classes, predicted_classes, average='macro')
#     print("Precision:", precision)
# except:
#     print("precision fail")

# try:
#     recall = recall_score(true_classes, predicted_classes, average='macro')
#     print("Recall:", recall)
# except:
#     print("recall fail")

# try:
#     f1 = f1_score(true_classes, predicted_classes, average='macro')
#     print("F1 Score:", f1)
# except:
#     print("f1 fail")

# try:
#     class_report = classification_report(true_classes, predicted_classes)
#     print("Classification Report:")
#     print(class_report)
# except:
#     print("class report fail")

### Model Constructor

In [51]:
# def build_vgg_19_model(
#     image_size,
#     label_class,
#     weights
# ):
#     # Input layers for color and depth images
#     color_input = Input(shape=(image_size[0], image_size[1], 3), name='color_input')
#     depth_input = Input(shape=(image_size[0], image_size[1], 1), name='depth_input')

#     # Base VGG19 model for color images
#     base_color = VGG19(include_top=False, weights=weights, input_tensor=color_input)

#     # Base VGG19 model for depth images (weights set to None to initialize from scratch)
#     base_depth = VGG19(include_top=False, weights=None, input_tensor=depth_input)

#     # Flatten and concatenate the output from both VGG19 models
#     color_features = base_color.output
#     depth_features = base_depth.output
#     concatenated = Concatenate()([color_features, depth_features])
#     flattened = Flatten()(concatenated)

#     # Additional dense layers and output layer
#     X = layers.Dense(512, activation='relu')(flattened)
#     X = layers.Dropout(0.4)(X)
#     X = layers.BatchNormalization()(X)
#     X = layers.Dense(512, activation='relu')(X)
#     X = layers.Dropout(0.3)(X)
#     X = layers.BatchNormalization()(X)

#     # Output layer
#     preds = layers.Dense(label_class, activation='softmax')(X)

#     # Create and compile the model
#     model = Model(inputs=[color_input, depth_input], outputs=preds)
#     model.compile(optimizer=optimizers.RMSprop(learning_rate=LEARNINGRATE), loss='categorical_crossentropy', metrics=['categorical_accuracy'])

#     return model

In [52]:
def build_vgg_19_model(
    image_size,
    label_class,
    weights,
):
    # Loading the VGG models with unique names
    vgg_color = VGG19(include_top=False, weights=weights, input_shape=(image_size[0], image_size[1], 3))
    for layer in vgg_color.layers:
        layer._name = layer.name + "_color"   # rename the layers by appending "_color"

    vgg_depth = VGG19(include_top=False, weights=None, input_shape=(image_size[0], image_size[1], 1))
    for layer in vgg_depth.layers:
        layer._name = layer.name + "_depth"   # rename the layers by appending "_depth"

    # Extract features from both models
    color_features = vgg_color.output
    depth_features = vgg_depth.output

    # Concatenate the extracted features
    concatenated = Concatenate()([color_features, depth_features])

    # Flatten and add a dense layer (assuming a simple classification task)
    flattened = Flatten()(concatenated)

    outputs = Dense(label_class, activation='softmax')(flattened)
    model = Model(inputs=[vgg_color.input, vgg_depth.input], outputs=outputs)
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])
    return model

### DONE- VGG19 (batch size = 64; epoch = 15) => DEFAULT

In [53]:
# vgg19 = build_vgg_19_model(
#     image_size=IMAGE_SIZE,
#     label_class=LABEL_CLASS,
#     weights=WEIGHTS,
# )

# print("Beginning VGG19 training..")

# vgg19.fit(
#     [color_train, depth_train],
#     labels_train,
#     # validation_data=([color_val, depth_val], labels_val),
#     epochs=EPOCHS,
#     batch_size=BATCH_SIZE,
#     verbose=1,
#     callbacks=[
#         callbacks.EarlyStopping(patience=3),
#         callbacks.ModelCheckpoint("vgg_19_model_best.h5", save_best_only=True),  # Save the entire model
#         callbacks.ReduceLROnPlateau(monitor='val_loss', patience= 2, verbose= 1),      
#     ],
# )
# print("Ended VGG19 training.")

# # Save the entire model
# vgg19.save("vgg_19_model.keras")
# print("VGG19 model saved")

# predicted_labels = vgg19.predict([color_test, depth_test])
# predicted_classes = np.argmax(predicted_labels, axis=1)
# true_classes = np.argmax(labels_test, axis=1)
# try:
#     accuracy = accuracy_score(true_classes, predicted_classes)
#     print("Accuracy:", accuracy)
# except:
#     print("accuracy fail")

# try:
#     precision = precision_score(true_classes, predicted_classes, average='macro')
#     print("Precision:", precision)
# except:
#     print("precision fail")

# try:
#     recall = recall_score(true_classes, predicted_classes, average='macro')
#     print("Recall:", recall)
# except:
#     print("recall fail")

# try:
#     # f1 = f1_score(true_classes, predicted_classes, average='macro')
#     f1 = 2 * ((precision * recall) / (precision + recall))
#     print("F1 Score:", f1)
# except Exception as e:
#     print("f1 fail")
#     print(e)

# try:
#     class_report = classification_report(true_classes, predicted_classes)
#     print("Classification Report:")
#     print(class_report)
# except:
#     print("class report fail")

### DONE- VGG19 (batch size = 32; epoch = 15)

In [54]:
# vgg19 = build_vgg_19_model(
#     image_size=IMAGE_SIZE,
#     label_class=LABEL_CLASS,
#     weights=WEIGHTS,
# )

# print("Beginning VGG19 training..")

# vgg19.fit(
#     [color_train, depth_train],
#     labels_train,
#     # validation_data=([color_val, depth_val], labels_val),
#     epochs=EPOCHS,
#     batch_size=32,
#     verbose=1,
#     callbacks=[
#         callbacks.EarlyStopping(patience=3),
#         callbacks.ModelCheckpoint("vgg_19_model_best.h5", save_best_only=True),  # Save the entire model
#         callbacks.ReduceLROnPlateau(monitor='val_loss', patience= 2, verbose= 1),      
#     ],
# )
# print("Ended VGG19 training.")

# # Save the entire model
# vgg19.save("vgg_19_model.keras")
# print("VGG19 model saved")

# predicted_labels = vgg19.predict([color_test, depth_test])
# predicted_classes = np.argmax(predicted_labels, axis=1)
# true_classes = np.argmax(labels_test, axis=1)
# try:
#     accuracy = accuracy_score(true_classes, predicted_classes)
#     print("Accuracy:", accuracy)
# except:
#     print("accuracy fail")

# try:
#     precision = precision_score(true_classes, predicted_classes, average='macro')
#     print("Precision:", precision)
# except:
#     print("precision fail")

# try:
#     recall = recall_score(true_classes, predicted_classes, average='macro')
#     print("Recall:", recall)
# except:
#     print("recall fail")

# try:
#     # f1 = f1_score(true_classes, predicted_classes, average='macro')
#     f1 = 2 * ((precision * recall) / (precision + recall))
#     print("F1 Score:", f1)
# except Exception as e:
#     print("f1 fail")
#     print(e)

# try:
#     class_report = classification_report(true_classes, predicted_classes)
#     print("Classification Report:")
#     print(class_report)
# except:
#     print("class report fail")

### DOne - VGG19 (batch size = 64; epoch = 30)

In [55]:
# vgg19 = build_vgg_19_model(
#     image_size=IMAGE_SIZE,
#     label_class=LABEL_CLASS,
#     weights=WEIGHTS,
# )

# print("Beginning VGG19 training..")

# vgg19.fit(
#     [color_train, depth_train],
#     labels_train,
#     # validation_data=([color_val, depth_val], labels_val),
#     epochs=30,
#     batch_size=64,
#     verbose=1,
#     callbacks=[
#         callbacks.EarlyStopping(patience=3),
#         callbacks.ModelCheckpoint("vgg_19_model_best.h5", save_best_only=True),  # Save the entire model
#         callbacks.ReduceLROnPlateau(monitor='val_loss', patience= 2, verbose= 1),      
#     ],
# )
# print("Ended VGG19 training.")

# # Save the entire model
# # vgg19.save("vgg_19_model.keras")
# # print("VGG19 model saved")

# predicted_labels = vgg19.predict([color_test, depth_test])
# predicted_classes = np.argmax(predicted_labels, axis=1)
# true_classes = np.argmax(labels_test, axis=1)
# try:
#     accuracy = accuracy_score(true_classes, predicted_classes)
#     print("Accuracy:", accuracy)
# except:
#     print("accuracy fail")

# try:
#     precision = precision_score(true_classes, predicted_classes, average='macro')
#     print("Precision:", precision)
# except:
#     print("precision fail")

# try:
#     recall = recall_score(true_classes, predicted_classes, average='macro')
#     print("Recall:", recall)
# except:
#     print("recall fail")

# try:
#     # f1 = f1_score(true_classes, predicted_classes, average='macro')
#     f1 = 2 * ((precision * recall) / (precision + recall))
#     print("F1 Score:", f1)
# except Exception as e:
#     print("f1 fail")
#     print(e)

# try:
#     class_report = classification_report(true_classes, predicted_classes)
#     print("Classification Report:")
#     print(class_report)
# except:
#     print("class report fail")

### DONE- VGG19 (batch size = 64; epoch = 45)

In [56]:
# vgg19 = build_vgg_19_model(
#     image_size=IMAGE_SIZE,
#     label_class=LABEL_CLASS,
#     weights=WEIGHTS,
# )

# print("Beginning VGG19 training..")

# vgg19.fit(
#     [color_train, depth_train],
#     labels_train,
#     # validation_data=([color_val, depth_val], labels_val),
#     epochs=45,
#     batch_size=64,
#     verbose=1,
#     callbacks=[
#         callbacks.EarlyStopping(patience=3),
#         callbacks.ModelCheckpoint("vgg_19_model_best.h5", save_best_only=True),  # Save the entire model
#         callbacks.ReduceLROnPlateau(monitor='val_loss', patience= 2, verbose= 1),      
#     ],
# )
# print("Ended VGG19 training.")

# # Save the entire model
# # vgg19.save("vgg_19_model.keras")
# # print("VGG19 model saved")

# predicted_labels = vgg19.predict([color_test, depth_test])
# predicted_classes = np.argmax(predicted_labels, axis=1)
# true_classes = np.argmax(labels_test, axis=1)
# try:
#     accuracy = accuracy_score(true_classes, predicted_classes)
#     print("Accuracy:", accuracy)
# except:
#     print("accuracy fail")

# try:
#     precision = precision_score(true_classes, predicted_classes, average='macro')
#     print("Precision:", precision)
# except:
#     print("precision fail")

# try:
#     recall = recall_score(true_classes, predicted_classes, average='macro')
#     print("Recall:", recall)
# except:
#     print("recall fail")

# try:
#     # f1 = f1_score(true_classes, predicted_classes, average='macro')
#     f1 = 2 * ((precision * recall) / (precision + recall))
#     print("F1 Score:", f1)
# except Exception as e:
#     print("f1 fail")
#     print(e)

# try:
#     class_report = classification_report(true_classes, predicted_classes)
#     print("Classification Report:")
#     print(class_report)
# except:
#     print("class report fail")

## DONE - VGG19 VGG19 (batch size = 64; epoch = 15, Optimizer = nadam)

In [57]:
# def build_vgg_19_model_nadam(
#     image_size,
#     label_class,
#     weights,
# ):
#     # Loading the VGG models with unique names
#     # vgg_color = VGG19(include_top=False, weights=weights, input_tensor=color_input)
#     vgg_color = VGG19(include_top=False, weights=weights, input_shape=(image_size[0], image_size[1], 3))
#     for layer in vgg_color.layers:
#         layer._name = layer.name + "_color"   # rename the layers by appending "_color"

#     # vgg_depth = VGG19(include_top=False, weights=None, input_tensor=depth_input)
#     vgg_depth = VGG19(include_top=False, weights=None, input_shape=(image_size[0], image_size[1], 1))
#     for layer in vgg_depth.layers:
#         layer._name = layer.name + "_depth"   # rename the layers by appending "_depth"

#     # Extract features from both models
#     color_features = vgg_color.output
#     depth_features = vgg_depth.output

#     # Concatenate the extracted features
#     concatenated = Concatenate()([color_features, depth_features])

#     # Flatten and add a dense layer (assuming a simple classification task)
#     flattened = Flatten()(concatenated)

#         # Additional dense layers and output layer
#     X = layers.Dense(512, activation='relu')(flattened)
#     X = layers.Dropout(0.4)(X)
#     X = layers.BatchNormalization()(X)
#     X = layers.Dense(512, activation='relu')(X)
#     X = layers.Dropout(0.3)(X)
#     X = layers.BatchNormalization()(X)

#     outputs = Dense(label_class, activation='softmax')(X)
#     model = Model(inputs=[vgg_color.input, vgg_depth.input], outputs=outputs)
#     model.compile(optimizer='nadam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])
#     return model

In [58]:
# print("Beginning VGG19 training..")

# vgg19 = build_vgg_19_model_nadam(
#     image_size=IMAGE_SIZE,
#     label_class=LABEL_CLASS,
#     weights=WEIGHTS,
# )

# vgg19.fit(
#     [color_train, depth_train],
#     labels_train,
#     # validation_data=([color_val, depth_val], labels_val),
#     epochs=15,
#     batch_size=64,
#     verbose=0,
#     callbacks=[
#         callbacks.EarlyStopping(patience=3),
#         callbacks.ModelCheckpoint("vgg_19_model_best.h5", save_best_only=True),  # Save the entire model
#         callbacks.ReduceLROnPlateau(monitor='val_loss', patience= 2, verbose= 1),      
#     ],
# )

# print("Ended VGG19 training")

# predicted_labels = vgg19.predict([color_test, depth_test])
# predicted_classes = np.argmax(predicted_labels, axis=1)
# true_classes = np.argmax(labels_test, axis=1)
# try:
#     accuracy = accuracy_score(true_classes, predicted_classes)
#     print("Accuracy:", accuracy)
# except:
#     print("accuracy fail")

# try:
#     precision = precision_score(true_classes, predicted_classes, average='macro')
#     print("Precision:", precision)
# except:
#     print("precision fail")

# try:
#     recall = recall_score(true_classes, predicted_classes, average='macro')
#     print("Recall:", recall)
# except Exception as e:
#     print("recall fail")
#     print(e)

# try:
#     # f1 = f1_score(true_classes, predicted_classes, average='macro')
#     f1 = 2 * ((precision * recall) / (precision + recall))
#     print("F1 Score:", f1)
# except Exception as e:
#     print("f1 fail")
#     print(e)

# try:
#     class_report = classification_report(true_classes, predicted_classes)
#     print("Classification Report:")
#     print(class_report)
# except:
#     print("class report fail")


## Done - VGG19 VGG19 (batch size = 64; epoch = 15, Optimizer = SGD)

In [59]:
# def build_vgg_19_model_sgd(
#     image_size,
#     label_class,
#     weights,
# ):
#     # Loading the VGG models with unique names
#     # vgg_color = VGG19(include_top=False, weights=weights, input_tensor=color_input)
#     vgg_color = VGG19(include_top=False, weights=weights, input_shape=(image_size[0], image_size[1], 3))
#     for layer in vgg_color.layers:
#         layer._name = layer.name + "_color"   # rename the layers by appending "_color"

#     # vgg_depth = VGG19(include_top=False, weights=None, input_tensor=depth_input)
#     vgg_depth = VGG19(include_top=False, weights=None, input_shape=(image_size[0], image_size[1], 1))
#     for layer in vgg_depth.layers:
#         layer._name = layer.name + "_depth"   # rename the layers by appending "_depth"

#     # Extract features from both models
#     color_features = vgg_color.output
#     depth_features = vgg_depth.output

#     # Concatenate the extracted features
#     concatenated = Concatenate()([color_features, depth_features])

#     # Flatten and add a dense layer (assuming a simple classification task)
#     flattened = Flatten()(concatenated)

#     # Additional dense layers and output layer
#     X = layers.Dense(512, activation='relu')(flattened)
#     X = layers.Dropout(0.4)(X)
#     X = layers.BatchNormalization()(X)
#     X = layers.Dense(512, activation='relu')(X)
#     X = layers.Dropout(0.3)(X)
#     X = layers.BatchNormalization()(X)

#     outputs = Dense(label_class, activation='softmax')(X)
#     model = Model(inputs=[vgg_color.input, vgg_depth.input], outputs=outputs)
#     model.compile(optimizer='SGD', loss='categorical_crossentropy', metrics=['categorical_accuracy'])
#     return model

In [60]:
# print("Beginning VGG16 training..")

# vgg19 = build_vgg_19_model_sgd(
#     image_size=IMAGE_SIZE,
#     label_class=LABEL_CLASS,
#     weights=WEIGHTS,
# )

# vgg19.fit(
#     [color_train, depth_train],
#     labels_train,
#     # validation_data=([color_val, depth_val], labels_val),
#     epochs=15,
#     batch_size=64,
#     verbose=0,
#     callbacks=[
#         callbacks.EarlyStopping(patience=3),
#         callbacks.ModelCheckpoint("vgg_19_model_best.h5", save_best_only=True),  # Save the entire model
#         callbacks.ReduceLROnPlateau(monitor='val_loss', patience= 2, verbose= 1),      
#     ],
# )

# print("Ended VGG19 training")

# predicted_labels = vgg19.predict([color_test, depth_test])
# predicted_classes = np.argmax(predicted_labels, axis=1)
# true_classes = np.argmax(labels_test, axis=1)
# try:
#     accuracy = accuracy_score(true_classes, predicted_classes)
#     print("Accuracy:", accuracy)
# except:
#     print("accuracy fail")

# try:
#     precision = precision_score(true_classes, predicted_classes, average='macro')
#     print("Precision:", precision)
# except:
#     print("precision fail")

# try:
#     recall = recall_score(true_classes, predicted_classes, average='macro')
#     print("Recall:", recall)
# except Exception as e:
#     print("recall fail")
#     print(e)

# try:
#     # f1 = f1_score(true_classes, predicted_classes, average='macro')
#     f1 = 2 * ((precision * recall) / (precision + recall))
#     print("F1 Score:", f1)
# except Exception as e:
#     print("f1 fail")
#     print(e)

# try:
#     class_report = classification_report(true_classes, predicted_classes)
#     print("Classification Report:")
#     print(class_report)
# except:
#     print("class report fail")

## VGG19 VGG19 (batch size = 64; epoch = 15, Optimizer = RMSprop)¶

In [61]:
def build_vgg_19_model_rmsprop(
    image_size,
    label_class,
    weights,
):
    # Loading the VGG models with unique names
    # vgg_color = VGG19(include_top=False, weights=weights, input_tensor=color_input)
    vgg_color = VGG19(include_top=False, weights=weights, input_shape=(image_size[0], image_size[1], 3))
    for layer in vgg_color.layers:
        layer._name = layer.name + "_color"   # rename the layers by appending "_color"

    # vgg_depth = VGG19(include_top=False, weights=None, input_tensor=depth_input)
    vgg_depth = VGG19(include_top=False, weights=None, input_shape=(image_size[0], image_size[1], 1))
    for layer in vgg_depth.layers:
        layer._name = layer.name + "_depth"   # rename the layers by appending "_depth"

    # Extract features from both models
    color_features = vgg_color.output
    depth_features = vgg_depth.output

    # Concatenate the extracted features
    concatenated = Concatenate()([color_features, depth_features])

    # Flatten and add a dense layer (assuming a simple classification task)
    flattened = Flatten()(concatenated)

    # Additional dense layers and output layer
    X = layers.Dense(512, activation='relu')(flattened)
    X = layers.Dropout(0.4)(X)
    X = layers.BatchNormalization()(X)
    X = layers.Dense(512, activation='relu')(X)
    X = layers.Dropout(0.3)(X)
    X = layers.BatchNormalization()(X)

    outputs = Dense(label_class, activation='softmax')(X)
    model = Model(inputs=[vgg_color.input, vgg_depth.input], outputs=outputs)
    model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['categorical_accuracy'])
    return model

In [62]:
print("Beginning VGG19 training..")

vgg19 = build_vgg_19_model_rmsprop(
    image_size=IMAGE_SIZE,
    label_class=LABEL_CLASS,
    weights=WEIGHTS,
)

vgg19.fit(
    [color_train, depth_train],
    labels_train,
    # validation_data=([color_val, depth_val], labels_val),
    epochs=15,
    batch_size=64,
    verbose=0,
    callbacks=[
        callbacks.EarlyStopping(patience=3),
        callbacks.ModelCheckpoint("vgg_19_model_best.h5", save_best_only=True),  # Save the entire model
        callbacks.ReduceLROnPlateau(monitor='val_loss', patience= 2, verbose= 1),      
    ],
)

print("Ended VGG19 training")

predicted_labels = vgg19.predict([color_test, depth_test])
predicted_classes = np.argmax(predicted_labels, axis=1)
true_classes = np.argmax(labels_test, axis=1)
try:
    accuracy = accuracy_score(true_classes, predicted_classes)
    print("Accuracy:", accuracy)
except:
    print("accuracy fail")

try:
    precision = precision_score(true_classes, predicted_classes, average='macro')
    print("Precision:", precision)
except:
    print("precision fail")

try:
    recall = recall_score(true_classes, predicted_classes, average='macro')
    print("Recall:", recall)
except Exception as e:
    print("recall fail")
    print(e)

try:
    # f1 = f1_score(true_classes, predicted_classes, average='macro')
    f1 = 2 * ((precision * recall) / (precision + recall))
    print("F1 Score:", f1)
except Exception as e:
    print("f1 fail")
    print(e)

try:
    class_report = classification_report(true_classes, predicted_classes)
    print("Classification Report:")
    print(class_report)
except:
    print("class report fail")

Beginning VGG19 training..


Ended VGG19 training


  1/135 [..............................] - ETA: 1:18

  2/135 [..............................] - ETA: 10s 

  3/135 [..............................] - ETA: 10s

...

133/135 [============================>.] - ETA: 0s

134/135 [============================>.] - ETA: 0s

135/135 [==============================] - ETA: 0s

135/135 [==============================] - 11s 79ms/step


Accuracy: 0.9493055555555555
Precision: 0.9520429988918337
Recall: 0.9490071732279173
F1 Score: 0.9505226620740184
Classification Report:
              precision    recall  f1-score   support

           0       0.93      0.94      0.93       189
           1       0.97      0.98      0.98       175
           2       0.95      0.97      0.96       187
           3       0.96      0.94      0.95       172
           4       0.95      0.96      0.95       182
           5       0.99      0.98      0.98       186
           6       0.96      0.97      0.97       196
           7       0.97      0.93      0.95       184
           8       0.99      0.97      0.98       166
           9       0.96      0.98      0.97       187
          10       1.00      0.96      0.98       167
          11       0.98      0.99      0.99       177
          12       0.93      0.99      0.96       157
          13       0.97      0.86      0.91       181
          14       0.85      0.98      0.91       2

# Resnet model

### Preloaded Model

In [63]:
# resnet_preload_model = keras.models.load_model('resnet_model.keras')

In [64]:
# predicted_labels = resnet_preload_model.predict([color_test, depth_test])
# predicted_classes = np.argmax(predicted_labels, axis=1)
# true_classes = np.argmax(labels_test, axis=1)
# try:
#     accuracy = accuracy_score(true_classes, predicted_classes)
#     print("Accuracy:", accuracy)
# except:
#     print("accuracy fail")

# try:
#     precision = precision_score(true_classes, predicted_classes, average='macro')
#     print("Precision:", precision)
# except:
#     print("precision fail")

# try:
#     recall = recall_score(true_classes, predicted_classes, average='macro')
#     print("Recall:", recall)
# except:
#     print("recall fail")

# try:
#     f1 = f1_score(true_classes, predicted_classes, average='macro')
#     print("F1 Score:", f1)
# except:
#     print("f1 fail")

# try:
#     class_report = classification_report(true_classes, predicted_classes)
#     print("Classification Report:")
#     print(class_report)
# except:
#     print("class report fail")

### Model Constructor

In [65]:
def build_resnet_model(image_size, label_class, weights):
    # Loading the ResNet models with unique names
    resnet_color = ResNet50V2(
        include_top=False,
        weights=weights,
        input_shape=(image_size[0], image_size[1], 3),
        pooling=None,  # Remove the global average pooling layer
        classes=label_class,
        classifier_activation="softmax",
    )
    for layer in resnet_color.layers:
        layer._name = layer.name + "_color"  # Rename the layers by appending "_color"

    resnet_depth = ResNet50V2(
        include_top=False,
        weights=None,
        input_shape=(image_size[0], image_size[1], 1),
    )
    for layer in resnet_depth.layers:
        layer._name = layer.name + "_depth"  # Rename the layers by appending "_depth"

    # Add a Global Average Pooling layer to both models
    color_features = GlobalAveragePooling2D()(resnet_color.output)
    depth_features = GlobalAveragePooling2D()(resnet_depth.output)

    # Concatenate the extracted features
    concatenated = Concatenate()([color_features, depth_features])

    # Flatten and add a dense layer (assuming a simple classification task)
    flattened = Flatten()(concatenated)

    outputs = Dense(label_class, activation='softmax')(flattened)
    model = Model(inputs=[resnet_color.input, resnet_depth.input], outputs=outputs)
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

    return model

### DONE- ResNet (batch size = 64; epoch = 15) => DEFAULT

In [66]:
# resnet = build_resnet_model(
#     image_size=IMAGE_SIZE,
#     label_class=LABEL_CLASS,
#     weights='imagenet',
# )

# print("Beginning resnet training..")

# resnet.fit(
#     [color_train, depth_train],
#     labels_train,
#     # validation_data=([color_val, depth_val], labels_val),
#     epochs=EPOCHS,
#     batch_size=BATCH_SIZE,
#     verbose=1,
#     callbacks=[
#         callbacks.EarlyStopping(patience=3),
#         callbacks.ReduceLROnPlateau(monitor='categorical_crossentropy', patience= 2, verbose= 1),      
#         callbacks.ModelCheckpoint("resnet_model_best.h5", save_best_only=True),  # Save the entire model
#     ],
# )
# print("ended resnet training..")

# # Save the entire model
# # resnet.save("resnet_model.keras")
# # print("resnet model saved.")

# predicted_labels = resnet.predict([color_test, depth_test])
# predicted_classes = np.argmax(predicted_labels, axis=1)
# true_classes = np.argmax(labels_test, axis=1)
# try:
#     accuracy = accuracy_score(true_classes, predicted_classes)
#     print("Accuracy:", accuracy)
# except:
#     print("accuracy fail")

# try:
#     precision = precision_score(true_classes, predicted_classes, average='macro')
#     print("Precision:", precision)
# except:
#     print("precision fail")

# try:
#     recall = recall_score(true_classes, predicted_classes, average='macro')
#     print("Recall:", recall)
# except Exception as e:
#     print("recall fail")
#     print(e)

# try:
#     # f1 = f1_score(true_classes, predicted_classes, average='macro')
#     f1 = 2 * ((precision * recall) / (precision + recall))
#     print("F1 Score:", f1)
# except:
#     print("f1 fail")

# try:
#     class_report = classification_report(true_classes, predicted_classes)
#     print("Classification Report:")
#     print(class_report)
# except:
#     print("class report fail")

### DONE- ResNet (batch size = 32; epoch = 15)

In [67]:
# resnet = build_resnet_model(
#     image_size=IMAGE_SIZE,
#     label_class=LABEL_CLASS,
#     weights='imagenet',
# )

# print("Beginning resnet training..")

# resnet.fit(
#     [color_train, depth_train],
#     labels_train,
#     # validation_data=([color_val, depth_val], labels_val),
#     epochs=15,
#     batch_size=32,
#     verbose=1,
#     callbacks=[
#         callbacks.EarlyStopping(patience=3),
#         callbacks.ReduceLROnPlateau(monitor='categorical_crossentropy', patience= 2, verbose= 1),      
#         callbacks.ModelCheckpoint("resnet_model_best.h5", save_best_only=True),  # Save the entire model
#     ],
# )
# print("ended resnet training..")

# # Save the entire model
# # resnet.save("resnet_model.keras")
# # print("resnet model saved.")

# predicted_labels = resnet.predict([color_test, depth_test])
# predicted_classes = np.argmax(predicted_labels, axis=1)
# true_classes = np.argmax(labels_test, axis=1)
# try:
#     accuracy = accuracy_score(true_classes, predicted_classes)
#     print("Accuracy:", accuracy)
# except:
#     print("accuracy fail")

# try:
#     precision = precision_score(true_classes, predicted_classes, average='macro')
#     print("Precision:", precision)
# except:
#     print("precision fail")

# try:
#     recall = recall_score(true_classes, predicted_classes, average='macro')
#     print("Recall:", recall)
# except Exception as e:
#     print("recall fail")
#     print(e)

# try:
#     # f1 = f1_score(true_classes, predicted_classes, average='macro')
#     f1 = 2 * ((precision * recall) / (precision + recall))
#     print("F1 Score:", f1)
# except:
#     print("f1 fail")

# try:
#     class_report = classification_report(true_classes, predicted_classes)
#     print("Classification Report:")
#     print(class_report)
# except:
#     print("class report fail")

### DONE- ResNet (batch size = 64; epoch = 30)

In [68]:
# resnet = build_resnet_model(
#     image_size=IMAGE_SIZE,
#     label_class=LABEL_CLASS,
#     weights='imagenet',
# )

# print("Beginning resnet training..")

# resnet.fit(
#     [color_train, depth_train],
#     labels_train,
#     # validation_data=([color_val, depth_val], labels_val),
#     epochs=30,
#     batch_size=64,
#     verbose=1,
#     callbacks=[
#         callbacks.EarlyStopping(patience=3),
#         callbacks.ReduceLROnPlateau(monitor='categorical_crossentropy', patience= 2, verbose= 1),      
#         callbacks.ModelCheckpoint("resnet_model_best.h5", save_best_only=True),  # Save the entire model
#     ],
# )
# print("ended resnet training..")

# # Save the entire model
# # resnet.save("resnet_model.keras")
# # print("resnet model saved.")

# predicted_labels = resnet.predict([color_test, depth_test])
# predicted_classes = np.argmax(predicted_labels, axis=1)
# true_classes = np.argmax(labels_test, axis=1)
# try:
#     accuracy = accuracy_score(true_classes, predicted_classes)
#     print("Accuracy:", accuracy)
# except:
#     print("accuracy fail")

# try:
#     precision = precision_score(true_classes, predicted_classes, average='macro')
#     print("Precision:", precision)
# except:
#     print("precision fail")

# try:
#     recall = recall_score(true_classes, predicted_classes, average='macro')
#     print("Recall:", recall)
# except Exception as e:
#     print("recall fail")
#     print(e)

# try:
#     # f1 = f1_score(true_classes, predicted_classes, average='macro')
#     f1 = 2 * ((precision * recall) / (precision + recall))
#     print("F1 Score:", f1)
# except:
#     print("f1 fail")

# try:
#     class_report = classification_report(true_classes, predicted_classes)
#     print("Classification Report:")
#     print(class_report)
# except:
#     print("class report fail")

### DONE- ResNet (batch size = 64; epoch = 45)

In [69]:
# resnet = build_resnet_model(
#     image_size=IMAGE_SIZE,
#     label_class=LABEL_CLASS,
#     weights='imagenet',
# )

# print("Beginning resnet training..")

# resnet.fit(
#     [color_train, depth_train],
#     labels_train,
#     # validation_data=([color_val, depth_val], labels_val),
#     epochs=45,
#     batch_size=64,
#     verbose=1,
#     callbacks=[
#         callbacks.EarlyStopping(patience=3),
#         callbacks.ReduceLROnPlateau(monitor='categorical_crossentropy', patience= 2, verbose= 1),      
#         callbacks.ModelCheckpoint("resnet_model_best.h5", save_best_only=True),  # Save the entire model
#     ],
# )
# print("ended resnet training..")

# # Save the entire model
# # resnet.save("resnet_model.keras")
# # print("resnet model saved.")

# predicted_labels = resnet.predict([color_test, depth_test])
# predicted_classes = np.argmax(predicted_labels, axis=1)
# true_classes = np.argmax(labels_test, axis=1)
# try:
#     accuracy = accuracy_score(true_classes, predicted_classes)
#     print("Accuracy:", accuracy)
# except:
#     print("accuracy fail")

# try:
#     precision = precision_score(true_classes, predicted_classes, average='macro')
#     print("Precision:", precision)
# except:
#     print("precision fail")

# try:
#     recall = recall_score(true_classes, predicted_classes, average='macro')
#     print("Recall:", recall)
# except Exception as e:
#     print("recall fail")
#     print(e)

# try:
#     # f1 = f1_score(true_classes, predicted_classes, average='macro')
#     f1 = 2 * ((precision * recall) / (precision + recall))
#     print("F1 Score:", f1)
# except:
#     print("f1 fail")

# try:
#     class_report = classification_report(true_classes, predicted_classes)
#     print("Classification Report:")
#     print(class_report)
# except:
#     print("class report fail")

## ResNet ResNet (batch size = 64; epoch = 15, Optimizer = nadam)¶

In [70]:
def build_resnet_model_nadam(
    image_size,
    label_class,
    weights,
):
    # Loading the ResNet models with unique names
    resnet_color = ResNet50V2(include_top=False, weights=weights, input_shape=(image_size[0], image_size[1], 3), 
                           pooling=None, classes=label_class, classifier_activation="softmax")
    for layer in resnet_color.layers:
        layer._name = layer.name + "_color"  # Rename the layers by appending "_color"

    resnet_depth = ResNet50V2(include_top=False, weights=None, input_shape=(image_size[0], image_size[1], 1))
    for layer in resnet_depth.layers:
        layer._name = layer.name + "_depth"  # Rename the layers by appending "_depth"

    # Extract features from both models
    # color_features = resnet_color.output
    # depth_features = resnet_depth.output

    # Add a Global Average Pooling layer to both models
    color_features = GlobalAveragePooling2D()(resnet_color.output)
    depth_features = GlobalAveragePooling2D()(resnet_depth.output)

    # Concatenate the extracted features
    concatenated = Concatenate()([color_features, depth_features])

    # Flatten and add a dense layer (assuming a simple classification task)
    flattened = Flatten()(concatenated)

    outputs = Dense(label_class, activation='softmax')(flattened)
    model = Model(inputs=[resnet_color.input, resnet_depth.input], outputs=outputs)
    model.compile(optimizer='nadam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])
    return model

In [71]:
print("Beginning ResNet training..")

resnet = build_resnet_model_nadam(
    image_size=IMAGE_SIZE,
    label_class=LABEL_CLASS,
    weights=WEIGHTS,
)

resnet.fit(
    [color_train, depth_train],
    labels_train,
    # validation_data=([color_val, depth_val], labels_val),
    epochs=15,
    batch_size=64,
    verbose=1,
    callbacks=[
        callbacks.EarlyStopping(patience=3),
        callbacks.ModelCheckpoint("resnet_model_best.h5", save_best_only=True),  # Save the entire model
        callbacks.ReduceLROnPlateau(monitor='val_loss', patience= 2, verbose= 1),      
    ],
)

print("Ended ResNet training")

predicted_labels = resnet.predict([color_test, depth_test])
predicted_classes = np.argmax(predicted_labels, axis=1)
true_classes = np.argmax(labels_test, axis=1)
try:
    accuracy = accuracy_score(true_classes, predicted_classes)
    print("Accuracy:", accuracy)
except:
    print("accuracy fail")

try:
    precision = precision_score(true_classes, predicted_classes, average='macro')
    print("Precision:", precision)
except:
    print("precision fail")

try:
    recall = recall_score(true_classes, predicted_classes, average='macro')
    print("Recall:", recall)
except Exception as e:
    print("recall fail")
    print(e)

try:
    # f1 = f1_score(true_classes, predicted_classes, average='macro')
    f1 = 2 * ((precision * recall) / (precision + recall))
    print("F1 Score:", f1)
except Exception as e:
    print("f1 fail")
    print(e)

try:
    class_report = classification_report(true_classes, predicted_classes)
    print("Classification Report:")
    print(class_report)
except:
    print("class report fail")


Beginning ResNet training..


Epoch 1/15


  1/158 [..............................] - ETA: 4:08:18 - loss: 3.4906 - categorical_accuracy: 0.0156

  2/158 [..............................] - ETA: 42s - loss: 3.2127 - categorical_accuracy: 0.1016    

  3/158 [..............................] - ETA: 41s - loss: 3.1054 - categorical_accuracy: 0.1458

...

131/135 [============================>.] - ETA: 0s

133/135 [============================>.] - ETA: 0s

135/135 [==============================] - ETA: 0s

135/135 [==============================] - 8s 44ms/step


Accuracy: 0.9791666666666666
Precision: 0.9815601352683769
Recall: 0.9792568266098652
F1 Score: 0.9804071281277982
Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       189
           1       1.00      0.98      0.99       175
           2       0.99      0.96      0.98       187
           3       0.97      0.97      0.97       172
           4       0.99      0.99      0.99       182
           5       1.00      0.99      1.00       186
           6       0.99      0.97      0.98       196
           7       1.00      0.95      0.97       184
           8       1.00      0.99      1.00       166
           9       1.00      1.00      1.00       187
          10       0.98      0.99      0.99       167
          11       0.99      1.00      0.99       177
          12       1.00      0.99      0.99       157
          13       1.00      0.99      1.00       181
          14       0.95      1.00      0.97       2

## ResNet ResNet (batch size = 64; epoch = 15, Optimizer = SGD)¶

In [72]:
def build_resnet_model_sgd(
    image_size,
    label_class,
    weights,
):
    # Loading the VGG models with unique names
    resnet_color = ResNet50V2(include_top=False, weights=weights, input_shape=(image_size[0], image_size[1], 3), 
                           pooling=None, classes=label_class, classifier_activation="softmax")
    for layer in resnet_color.layers:
        layer._name = layer.name + "_color"  # Rename the layers by appending "_color"

    resnet_depth = ResNet50V2(include_top=False, weights=None, input_shape=(image_size[0], image_size[1], 1))
    for layer in resnet_depth.layers:
        layer._name = layer.name + "_depth"  # Rename the layers by appending "_depth"

    # Extract features from both models
    # color_features = resnet_color.output
    # depth_features = resnet_depth.output
    
    # Add a Global Average Pooling layer to both models
    color_features = GlobalAveragePooling2D()(resnet_color.output)
    depth_features = GlobalAveragePooling2D()(resnet_depth.output)

    # Concatenate the extracted features
    concatenated = Concatenate()([color_features, depth_features])

    # Flatten and add a dense layer (assuming a simple classification task)
    flattened = Flatten()(concatenated)

    outputs = Dense(label_class, activation='softmax')(flattened)
    model = Model(inputs=[resnet_color.input, resnet_depth.input], outputs=outputs)
    model.compile(optimizer='SGD', loss='categorical_crossentropy', metrics=['categorical_accuracy'])
    return model

In [ ]:
print("Beginning ResNet training..")

resnet = build_resnet_model_sgd(
    image_size=IMAGE_SIZE,
    label_class=LABEL_CLASS,
    weights=WEIGHTS,
)

resnet.fit(
    [color_train, depth_train],
    labels_train,
    # validation_data=([color_val, depth_val], labels_val),
    epochs=15,
    batch_size=64,
    verbose=1,
    callbacks=[
        callbacks.EarlyStopping(patience=3),
        callbacks.ModelCheckpoint("resnet_model_best.h5", save_best_only=True),  # Save the entire model
        callbacks.ReduceLROnPlateau(monitor='val_loss', patience= 2, verbose= 1),      
    ],
)

print("Ended ResNet training")

predicted_labels = resnet.predict([color_test, depth_test])
predicted_classes = np.argmax(predicted_labels, axis=1)
true_classes = np.argmax(labels_test, axis=1)
try:
    accuracy = accuracy_score(true_classes, predicted_classes)
    print("Accuracy:", accuracy)
except:
    print("accuracy fail")

try:
    precision = precision_score(true_classes, predicted_classes, average='macro')
    print("Precision:", precision)
except:
    print("precision fail")

try:
    recall = recall_score(true_classes, predicted_classes, average='macro')
    print("Recall:", recall)
except Exception as e:
    print("recall fail")
    print(e)

try:
    # f1 = f1_score(true_classes, predicted_classes, average='macro')
    f1 = 2 * ((precision * recall) / (precision + recall))
    print("F1 Score:", f1)
except Exception as e:
    print("f1 fail")
    print(e)

try:
    class_report = classification_report(true_classes, predicted_classes)
    print("Classification Report:")
    print(class_report)
except:
    print("class report fail")

## ResNet ResNet (batch size = 64; epoch = 15, Optimizer = RMSprop)¶

In [ ]:
def build_resnet_model_rmsprop(
    image_size,
    label_class,
    weights,
):
    # Loading the VGG models with unique names
    resnet_color = ResNet50V2(include_top=False, weights=weights, input_shape=(image_size[0], image_size[1], 3), 
                           pooling=None, classes=label_class, classifier_activation="softmax")
    for layer in resnet_color.layers:
        layer._name = layer.name + "_color"  # Rename the layers by appending "_color"

    resnet_depth = ResNet50V2(include_top=False, weights=None, input_shape=(image_size[0], image_size[1], 1))
    for layer in resnet_depth.layers:
        layer._name = layer.name + "_depth"  # Rename the layers by appending "_depth"

    # Extract features from both models
    # color_features = resnet_color.output
    # depth_features = resnet_depth.output

    # Add a Global Average Pooling layer to both models
    color_features = GlobalAveragePooling2D()(resnet_color.output)
    depth_features = GlobalAveragePooling2D()(resnet_depth.output)

    # Concatenate the extracted features
    concatenated = Concatenate()([color_features, depth_features])

    # Flatten and add a dense layer (assuming a simple classification task)
    flattened = Flatten()(concatenated)
    
    outputs = Dense(label_class, activation='softmax')(flattened)
    model = Model(inputs=[resnet_color.input, resnet_depth.input], outputs=outputs)
    model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['categorical_accuracy'])
    return model

In [ ]:
print("Beginning ResNet training..")

resnet = build_resnet_model_rmsprop(
    image_size=IMAGE_SIZE,
    label_class=LABEL_CLASS,
    weights=WEIGHTS,
)

resnet.fit(
    [color_train, depth_train],
    labels_train,
    # validation_data=([color_val, depth_val], labels_val),
    epochs=15,
    batch_size=64,
    verbose=1,
    callbacks=[
        callbacks.EarlyStopping(patience=3),
        callbacks.ModelCheckpoint("resnet_model_best.h5", save_best_only=True),  # Save the entire model
        callbacks.ReduceLROnPlateau(monitor='val_loss', patience= 2, verbose= 1),      
    ],
)

print("Ended ResNet training")

predicted_labels = resnet.predict([color_test, depth_test])
predicted_classes = np.argmax(predicted_labels, axis=1)
true_classes = np.argmax(labels_test, axis=1)
try:
    accuracy = accuracy_score(true_classes, predicted_classes)
    print("Accuracy:", accuracy)
except:
    print("accuracy fail")

try:
    precision = precision_score(true_classes, predicted_classes, average='macro')
    print("Precision:", precision)
except:
    print("precision fail")

try:
    recall = recall_score(true_classes, predicted_classes, average='macro')
    print("Recall:", recall)
except Exception as e:
    print("recall fail")
    print(e)

try:
    # f1 = f1_score(true_classes, predicted_classes, average='macro')
    f1 = 2 * ((precision * recall) / (precision + recall))
    print("F1 Score:", f1)
except Exception as e:
    print("f1 fail")
    print(e)

try:
    class_report = classification_report(true_classes, predicted_classes)
    print("Classification Report:")
    print(class_report)
except:
    print("class report fail")